<div align='center'>
    <h2>Controle Quântico Ótimo</h2>
    <h4>Utilização do Algoritmo Genético para obtenção dos valores de controle</h4>
</div>

<div align='justify'>
    <p>Para que haja um controle ótimo de uma partícula, supondo que esta esteja em um estado inicial, é necessário formas de controle suficientemente eficientes já que a particula é a função de onda da partícula é definida em função do tempo. Para que isso seja possível utiliza-se a eficiência do algoritmo genético que pode usar ou não utilizar-se do elitismo juntamente com o controle preditivo, MPC. </p>
    <p>Abaixo são revelados os parâmetros para esse experimento, do qual podem ser ajustados:</p>
</div>

In [ ]:
# Bilbiotecas para auxílio na programação matemática
import math, sys 
import numpy as np
import sympy as sp
import cmath

from scipy import sparse # Produção das diagonais das matrizes
from scipy.sparse import diags 

# Plotagem 2D e 3D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm


from os import path # Suficiente para manipulação de arquivos
    
# Para solução exata
from scipy.special import hermite
from math import factorial

%matplotlib inline
count = 0

# Para otimização dos sistemas
from scipy import optimize
from random import randint, uniform, random

In [ ]:
## CONSTANTES PARA O CONTROLADOR

# Considerações: constante de Planck verdadeira: 1, massa: 1

TEMPO_ANALISE = 10 
QUANTIDADE_PONTOS_AMOSTRAGEM = 50
PASSO = TEMPO_ANALISE/QUANTIDADE_PONTOS_AMOSTRAGEM

MIN_HORIZONTE = 3 # Horizonte mínimo da análise
MAX_HORIZONTE = 40 # Horizonte máximo da análise

ONDA_DESEJADA = [[complex(1/np.sqrt(2),0.0)],[complex(1/np.sqrt(2),0.0)]] # [[c1];[c2]] = [[0.707],[0.707]]

# Determinação dos valores de início e fim da análise
num = PASSO
contador = 0
while num < 1:
    num *= 10
    contador += 1

INICIO_ANALISE = 0*10**(-contador) # Tempo inicial da analise em um horizonte
FINAL_ANALISE = (1*10**(-contador))+PASSO # Tempo final da analise em um horizonte

In [ ]:
## CONSTANTES PARA O ALGORITMO GENÉTICO

ELITISMO = True
PORCENTAGEM_ELITISMO = 0.20

TAMANHO_POP = 50
TAXA_MUTACAO = 0.10
TAXA_CRUZAMENTO = 0.70
GERACOES = 100

# Valores mínimos e máximos para gerar uma população
MIN = 0
MAX = 5

In [ ]:
# Manipulação das matrizes, soma e subtração

def somar(A, B):
    C = []
    num_linhas_a = len(A)
    num_colunas_a = len(A[0])
    
    for i in range (num_linhas_a):
        linha = [0]*num_colunas_a
        C.append(linha)
        for j in range(num_colunas_a):
            C[i][j] = A[i][j] + B[i][j]

    return C

def sub(A, B):
    C = []
    num_linhas_a = len(A)
    num_colunas_a = len(A[0])
    
    for i in range (num_linhas_a):
        linha = [0]*num_colunas_a
        C.append(linha)
        for j in range(num_linhas_a):
            C[i][j] = A[i][j] - B[i][j]

    return C

<div align='justify'>
    <h4>Runge-Kutta de quarta ordem</h4>
    <p>Nos procedimentos com estados quânticos, é necessário utilizar-se da equação de Schrödinger com o o objetivo de avaliar o valor da função de onda com o tempo. Sabe-se que sua equação se dá pela função de onda relacionada com a posição e tempo, no entanto, utilizando a notação de Dirac, a posição se torna um atributo irrelevante para a análise já que analisaremos níveis. Portanto: </p>
</div>

$$\frac{\partial{\ket{\psi(t)}}}{\partial{t}} = -iH\ket{\psi(t)}$$


In [ ]:
## psi' = -i * H * ket{psi}
def dpsi_dt(t, psi, H): # A derivada da onda em relação ao tempo não tem dependência temporal
    A = np.zeros((2,2), dtype=np.complex_)
    A = np.dot(complex(0,1),H) # i * H
    return -1*np.matmul(A,psi) # - i * H * ket{psi}

# Runge-Kutta de quarta ordem

def runge_kutta(onda, fator_runge_kutta, hamiltoniano, tempo_inicial = 0):
    
    # onda = [[c0],[c1]]
    
    k1 = dpsi_dt(tempo_inicial, onda, hamiltoniano)
    k2 = dpsi_dt(tempo_inicial + 0.5 * fator_runge_kutta, somar(onda, np.dot((0.5*fator_runge_kutta), k1)), hamiltoniano)
    k3 = dpsi_dt(tempo_inicial + 0.5 * fator_runge_kutta, somar(onda, np.dot((0.5*fator_runge_kutta), k2)), hamiltoniano)
    k4 = dpsi_dt(tempo_inicial + fator_runge_kutta, somar(onda, np.dot(fator_runge_kutta, k3)), hamiltoniano)
    
    ## y(i+1) = y(i) + h/6*(k1+2*k2+2*k3+k4)
    
    A = somar(np.dot(2,k3), k4)
    B = somar(np.dot(2,k2), k1)
    C = somar(A, B)

    runge = somar(onda,np.dot((fator_runge_kutta / 6.0),(C)))
        
    return runge

<div align='justify'>
    <h4>Estabelecimento da função objetivo</h4>
    <p>O estudo se baseia em uma função de erro, da qual exije a comparação de um sistema de dois níveis em um certo tempo em relação ao seu desejado. Para que isso seja possível estabelece-se a noção de controle preditivo.</p>
    <p>O controle preditivo é um modelo que se utiliza da avaliação futura para reajustar uma curva em relação a sua referência, ou seja, esta família de controles tem a capacidade de visualizar instantes a frente para definir qual é a melhor rota. Com esse tipo de controle, a função objetivo estabelece-se da seguinte forma:</p>
</div>

$$\argmin ||\sum^{i+h}_{j = i} \ket{\psi(j)} - \ket{\psi_d(j)}||$$

In [ ]:
def func_objetivo(x, it, onda_desejada, hamiltoniano, 
                  onda_inicial, horizonte, fator_runge_kutta):
    
    fo = 0
    runge = np.zeros((2,1),dtype=np.complex_)
        
    tempo = it   
    
    tempo_inicial = INICIO_ANALISE
    tempo_final = FINAL_ANALISE
    
    controles = np.zeros((2,2), dtype=np.complex_)
    controles = [[0, x[0]], [x[0], 0]]
    
    matriz_inicial = np.zeros((2,1),dtype=np.complex_)
    matriz_inicial = [[onda_inicial[0][0]], [onda_inicial[1][0]]] 

    matriz_desejada = np.zeros((2,1),dtype=np.complex_)
    
    ## Função-Objetivo (Return) = somatorio ||(Matriz_Origem - Matriz_Destino)||^2
    
    ## Avanço temporal
    
    matriz_desejada[0][0] = onda_desejada[0][0]*np.exp(-1*complex(0,1)
                                                       *(1/2*np.pi)*tempo)
    matriz_desejada[1][0] = onda_desejada[1][0]*np.exp(-1*complex(0,1)
                                                       *(3/2*np.pi)*tempo)

    fo += (np.linalg.norm(matriz_inicial-matriz_desejada))**2

    runge = runge_kutta(matriz_inicial, fator_runge_kutta, 
                        somar(hamiltoniano, controles), 
                        tempo_final = tempo_final, 
                        tempo_inicial = tempo_inicial)

    matriz_inicial = runge

    tempo_inicial += PASSO
    tempo_final += PASSO
    tempo += PASSO
    
    while horizonte > 1:
        
        matriz_desejada[0][0] = onda_desejada[0][0]*np.exp(-1*complex(0,1)
                                                           *(1/2*np.pi)*
                                                           tempo)
        matriz_desejada[1][0] = onda_desejada[1][0]*np.exp(
            -1*complex(0,1)*
            (3/2*np.pi)*tempo)
        
        fo += (np.linalg.norm(matriz_inicial-matriz_desejada))**2
        
        controles = [[0, x[len(x)-horizonte+1]], 
                     [x[len(x)-horizonte+1], 0]]

        runge = runge_kutta(matriz_inicial, fator_runge_kutta, somar
                            (hamiltoniano, controles), 
                            tempo_final = tempo_final, 
                            tempo_inicial = tempo_inicial)

        matriz_inicial = runge

        tempo_inicial += PASSO
        tempo_final += PASSO
        tempo += PASSO
        horizonte -= 1
    
    return fo

<div align='justify'>
<p>Para o funcionamento do MPC, cria-se um looping de forma a obter o melhor resultado para o ajuste da curva. Com esse valor em mãos, utiliza-se para os pontos da próxima iteração do método de controle além de efetivar o ajuste realizando o Runge-Kutta com o valor otimizado.</p>
</div>